In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.optimize import least_squares

# Data Preparation

In [95]:
swaption_df = pd.read_excel('IR Data.xlsx', sheet_name='Swaption', skiprows=2)
swaption_df_pivoted = swaption_df.melt(id_vars=["Expiry", "Tenor"], var_name="delta", value_name="impliedVol")

In [96]:
par_swap_rate_df = pd.read_csv("par_swap_rate.csv")
par_swap_rate_df = par_swap_rate_df.rename(columns={"Unnamed: 0": "Expiry"})
par_swap_rate_df_pivoted = par_swap_rate_df.melt(id_vars="Expiry", var_name="Tenor", value_name="parSwap")

In [97]:
data_df = swaption_df_pivoted.merge(par_swap_rate_df_pivoted, on = ["Expiry", "Tenor"], how = "left")

In [98]:
def generate_swap_delta(x):
    if x == "ATM":
        delta = 0
    else:
        delta = int(x.split("bps")[0]) * 0.0001
    return delta

In [99]:
data_df["impliedVol"] = data_df["impliedVol"] / 100
data_df["swapDelta"] = data_df["delta"].apply(lambda x: generate_swap_delta(x))
data_df["strike"] = data_df["parSwap"] + data_df["swapDelta"]
data_df["Expiry_no"] = data_df["Expiry"].apply(lambda x: int(x.split("Y")[0]))

In [100]:
data_df

,Expiry,Tenor,delta,impliedVol,parSwap,swapDelta,strike,Expiry_no
0,1Y,1Y,-200bps,0.9157,0.032007,-0.02,0.012007,1
1,1Y,2Y,-200bps,0.8327,0.033260,-0.02,0.013260,1
2,1Y,3Y,-200bps,0.7392,0.034011,-0.02,0.014011,1
3,1Y,5Y,-200bps,0.5519,0.035256,-0.02,0.015256,1
4,1Y,10Y,-200bps,0.4118,0.038428,-0.02,0.018428,1
...,...,...,...,...,...,...,...,...
160,10Y,1Y,+200bps,0.2578,0.042189,0.02,0.062189,10
161,10Y,2Y,+200bps,0.2571,0.043116,0.02,0.063116,10
162,10Y,3Y,+200bps,0.2537,0.044097,0.02,0.064097,10
163,10Y,5Y,+200bps,0.2280,0.046249,0.02,0.066249,10


# SABR

In [101]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [102]:
beta = 0.9

def sabrcalibration(x, strikes, vols, F, T):
    err = 0.0
    for i, vol in enumerate(vols):
        err += (vol - SABR(F, strikes[i], T,
                           x[0], beta, x[1], x[2]))**2

    return err

In [229]:
tenor_list = data_df['Tenor'].unique().tolist()
expiry_list = data_df['Expiry'].unique().tolist()
results = []

for expiry in expiry_list:
    for tenor in tenor_list:
        calibrate_df = data_df[(data_df['Expiry'] == expiry) & (data_df['Tenor'] == tenor)].copy()
        T = calibrate_df['Expiry_no'].iloc[0]
        r = 0
        F = calibrate_df['parSwap'].iloc[0]

        initialGuess = [0.2, -0.5, 1.5]
        res = least_squares(lambda x: sabrcalibration(x,
                                                    calibrate_df['strike'].values,
                                                    calibrate_df['impliedVol'].values,
                                                    F,
                                                    T),
                            initialGuess,
                            bounds=([0,-1,0],[np.inf,1,np.inf])
        )
        alpha = res.x[0]
        rho = res.x[1]
        nu = res.x[2]

        temp = [expiry, tenor, alpha, rho, nu]
        results.append(temp)

In [230]:
sabr_results_df = pd.DataFrame(results)
sabr_results_df.columns = ["Expiry", "Tenor", "Alpha", "Rho", "Nu"]

In [232]:
alpha_df = sabr_results_df[["Expiry","Tenor", "Alpha"]].copy()
alpha_df["Tenor"] = alpha_df["Tenor"].apply(lambda x: int(x.split("Y")[0]))

original_expiry_order = alpha_df["Expiry"].unique()
original_tenor_order = alpha_df["Tenor"].unique()
alpha_df.pivot(index="Expiry", columns="Tenor").reindex(index = original_expiry_order)

Alpha                                        
Tenor         1         2         3         5         10
Expiry                                                  
1Y      0.139071  0.184648  0.196838  0.177791  0.168437
5Y      0.166653  0.199549  0.210351  0.190099  0.172218
10Y     0.233541  0.302000  0.324386  0.361692  0.382548

In [233]:
rho_df = sabr_results_df[["Expiry","Tenor", "Rho"]].copy()
rho_df["Tenor"] = rho_df["Tenor"].apply(lambda x: int(x.split("Y")[0]))

original_expiry_order = rho_df["Expiry"].unique()
original_tenor_order = rho_df["Tenor"].unique()
rho_df.pivot(index="Expiry", columns="Tenor").reindex(index = original_expiry_order)

Rho                                        
Tenor         1         2         3         5         10
Expiry                                                  
1Y     -0.633209 -0.525119 -0.482809 -0.412381 -0.240901
5Y     -0.585862 -0.547110 -0.549799 -0.506429 -0.395731
10Y    -0.642604 -0.652417 -0.648831 -0.657004 -0.668822

In [234]:
nu_df = sabr_results_df[["Expiry","Tenor", "Nu"]].copy()
nu_df["Tenor"] = nu_df["Tenor"].apply(lambda x: int(x.split("Y")[0]))

original_expiry_order = nu_df["Expiry"].unique()
original_tenor_order = nu_df["Tenor"].unique()
nu_df.pivot(index="Expiry", columns="Tenor").reindex(index = original_expiry_order)

Nu                                        
Tenor         1         2         3         5         10
Expiry                                                  
1Y      2.049459  1.677431  1.438253  1.067301  0.810787
5Y      1.340758  1.062093  0.936756  0.675717  0.530926
10Y     1.220898  1.316304  1.275942  1.273556  1.098726

In [87]:
sabrvols = []
for K in calibrate_df['strike']:
    sabrvols.append(SABR(F, K, T, alpha, beta, rho, nu))

plt.figure(tight_layout=True)
plt.plot(calibrate_df['strike'], calibrate_df['impliedVol'], 'gs', label='Market Vols')
plt.plot(calibrate_df['strike'], sabrvols, 'm--', label='SABR Vols')
plt.legend()
plt.show()